In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['keras-pretrained-models', 'kermany2018']


In [2]:
Training_Dir = '../input/kermany2018/oct2017/OCT2017 /train'
Validation_Dir = '../input/kermany2018/oct2017/OCT2017 /val'
Testing_Dir = '../input/kermany2018/oct2017/OCT2017 /test'

In [3]:
from tensorflow.keras.applications.resnet50 import ResNet50
pre_trained_model = ResNet50(input_shape = (150, 150, 3), weights = None,include_top = False)
pre_trained_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 156, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 75, 75, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalizationV1) (None, 75, 75, 64)   256         conv1[0][0]                      
_____________________________________

/opt/conda/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [4]:
weights_file = '../input/keras-pretrained-models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model.load_weights(weights_file)

In [5]:
len(pre_trained_model.layers)

175

In [6]:
last_layer = pre_trained_model.get_layer('activation_48')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 5, 5, 2048)


In [7]:
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (4, activation='softmax')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 156, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 75, 75, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalizationV1) (None, 75, 75, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [9]:
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

training_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
validation_datagen = ImageDataGenerator(rescale = 1./255)
testing_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	Training_Dir,
	target_size=(150,150),
  shuffle=True,
	class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
	Validation_Dir,
	target_size=(150,150),
  shuffle=True,
	class_mode='categorical'
)

testing_generator = testing_datagen.flow_from_directory(
	Testing_Dir,
	target_size=(150,150),
  shuffle=True,
	class_mode='categorical'
)

Found 83484 images belonging to 4 classes.
Found 32 images belonging to 4 classes.
Found 968 images belonging to 4 classes.


In [10]:
history = model.fit_generator(train_generator, epochs=0, validation_data = testing_generator, verbose = 1)

Instructions for updating:
Use tf.cast instead.


In [11]:
history.history['acc'], history.history['val_acc'], history.history['loss'], history.history['val_loss']

KeyError: 'acc'

In [12]:
training_datagen = ImageDataGenerator(rescale = 1./255.)

train_generator = training_datagen.flow_from_directory(
	Training_Dir,
	target_size=(150,150),
  shuffle=True,
	class_mode='categorical'
)
y_train_pred = model.predict_generator(train_generator)
y_train = train_generator.classes[train_generator.index_array]
y_train_pred = np.argmax(y_train_pred, axis=-1)
from sklearn.metrics import confusion_matrix
import itertools
cnf_matrix = confusion_matrix(y_train, y_train_pred)
cnf_matrix

Found 83484 images belonging to 4 classes.


array([[   57,     0, 37131,    17],
       [   14,     0, 11331,     3],
       [   16,     0,  8599,     1],
       [   40,     0, 26268,     7]])

In [13]:
from sklearn.metrics import classification_report
target_names = ['CNV', 'DME', 'DRUSEN', 'NORMAL']
print(classification_report(y_train, y_train_pred, target_names=target_names))

              precision    recall  f1-score   support

         CNV       0.45      0.00      0.00     37205
         DME       0.00      0.00      0.00     11348
      DRUSEN       0.10      1.00      0.19      8616
      NORMAL       0.25      0.00      0.00     26315

   micro avg       0.10      0.10      0.10     83484
   macro avg       0.20      0.25      0.05     83484
weighted avg       0.29      0.10      0.02     83484



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [14]:
y_test_pred = model.predict_generator(testing_generator)
y_test = testing_generator.classes[testing_generator.index_array]
y_test_pred = np.argmax(y_test_pred, axis=-1)
cnf_matrix = confusion_matrix(y_test, y_test_pred)
cnf_matrix

array([[  1,   0, 241,   0],
       [  0,   0, 242,   0],
       [  0,   0, 242,   0],
       [  0,   0, 242,   0]])

In [15]:
print(classification_report(y_test, y_test_pred, target_names=target_names))

              precision    recall  f1-score   support

         CNV       1.00      0.00      0.01       242
         DME       0.00      0.00      0.00       242
      DRUSEN       0.25      1.00      0.40       242
      NORMAL       0.00      0.00      0.00       242

   micro avg       0.25      0.25      0.25       968
   macro avg       0.31      0.25      0.10       968
weighted avg       0.31      0.25      0.10       968



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
